# Importing required Packages

In [1]:
import requests
from bs4 import BeautifulSoup
import csv
from textblob import TextBlob

In [45]:
import mysql.connector

# Mysql Database Connection

In [46]:
mydb = mysql.connector.connect(host='localhost', user='root', password='1519')
print(mydb)

# Create Database and Table

In [47]:
mc = mydb.cursor()
#mc.execute("create database if not exists task")
mc.execute("use task")

In [5]:
#mc.execute("create table if not exists reviews(rtitle tinytext, rtext longtext, storageSize int, colour varchar(15), verified varchar(3), sentiment varchar(10))")

# Getting HTML content of link  and parsing using html5lib parser

In [12]:
url = "https://www.amazon.in/Apple-New-iPhone-12-128GB/dp/B08L5TNJHG/"
header={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36','referer':'https://www.amazon.com/s?k=nike+shoes+men&crid=28WRS5SFLWWZ6&sprefix=nike%2Caps%2C357&ref=nb_sb_ss_organic-diversity_2_4'}
r = requests.get(url, headers=header)
soup = BeautifulSoup(r.content, 'html5lib')

In [ ]:
#soup.prettify()

# Getting link of review page

In [13]:
soup=BeautifulSoup(r.content)
link=[]
for i in soup.findAll("a",{'data-hook':"see-all-reviews-link-foot"}):
        link.append(i['href'])

In [14]:
link

['/New-Apple-iPhone-12-64GB/product-reviews/B08L5TGWD1/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews']

# Getting Text content out of HTML tags

In [26]:
review_text=[]
review_title=[]
verified = []
rating = []
        
for j in range(len(link)):
    for k in range(100):
        response=requests.get('https://www.amazon.in'+link[j]+'&pageNumber='+str(k), headers=header)
        soup=BeautifulSoup(response.content)
        for i in soup.findAll(name='a',class_="a-size-base a-link-normal review-title a-color-base review-title-content a-text-bold"):
            review_title.append(i.find("span").text)
        for i in soup.findAll("span",{'class':"a-size-base review-text review-text-content", 'data-hook':"review-body"}):
            review_text.append(i.text.replace("\n", "").strip())
        for i in soup.findAll("span",{'class':"a-size-mini a-color-state a-text-bold", 'data-hook':"avp-badge"}):
            if(i.text == "Verified Purchase"):
                verified.append('YES')
            else:
                review_text.append('NO')
        for i in soup.findAll('i',{'class':"a-icon a-icon-star a-star-4 review-rating", 'data-hook':"review-star-rating"}):
            rating.append(i.find("span").text[:3])

In [21]:
print(soup.find(name='div',class_="a-row a-spacing-top-medium product-variation-strip").find("span").text)

Colour: (PRODUCT)REDSize: 64GBPattern Name: iPhone 12


In [22]:
colour, size = "RED", "64"

In [27]:
len(review_text),len(review_title), len(verified)

(1000, 1000, 1000)

In [30]:
review_text[:5]

['SUMMERY: As a first time iPhone user, I would say one should definitely buy iPhone ONLY for premium built & premium hardware (will talk later) while the whole iOS experience is pretty overrated. iPhones isn\'t any exceptionally fast with normal apps like YouTube, Insta etc, Android phones are equally snappy with them in 2022.PROS:1. Impressive hardware, the heptics, the autobrightness & otherwise highly ignored things in Android are way too perfectly executed in iPhone and it DOES make a big difference in user experience!!2. The screen is AMAZING.. it doesn\'t have 90 or 120 hz refresh rate but it FEELS better than my Nord with 90 hz!!3. A14 bionic promises years of performance ahead.4. Battery life is somewhere in between good & great.5.6. The dimensions of phone, it\'s smaller than usual android phones and feels much more handy along with less weight.7. Overall build quality is great.8. Best audio output out there.9. Long term support.CONS:1. SLOW CHARGING. 20 W only, you\'ll be je

In [124]:
#for text in review_text:
#    text = text.replace("\n", "").strip()


# Function to analyze the sentiment (Postive, Negative, Average)
# TextBlob used for analysis

In [31]:
def sentimentAnalysis(review):
    if TextBlob(review).sentiment.polarity>0:
        return "Positive"
    if TextBlob(review).sentiment.polarity==0:
        return "Average"
    else:
        return "Negative"

In [32]:
sentiment = []
for review in review_text:
    sentiment.append(sentimentAnalysis(review))
    #print(sentimentAnalysis(review))

In [ ]:
for title in review_title:
    for text in review_text:
        for v in verified:
            for sent in sentiment:
                val=(title, text, size, colour, v, sent)
                mc.execute("insert into reviews(rtitle, rtext, storageSize, colour, verified, sentiment) values (%s, %s, %s, %s, %s, %s)", val)
                mydb.commit()

# Insert data into Database

# Fetch reviews

In [56]:
mc.execute("select * from reviews where rating = 4 limit 5")
fetch = mc.fetchall()

for x in fetch:
    print(x)
    print("\n")

('An overall good premium experience', "SUMMERY: As a first time iPhone user, I would say one should definitely buy iPhone ONLY for premium built & premium hardware (will talk later) while the whole iOS experience is pretty overrated. iPhones isn\\'t any exceptionally fast with normal apps like YouTube, Insta etc, Android phones are equally snappy with them in 2022.PROS:1. Impressive hardware, the heptics, the autobrightness & otherwise highly ignored things in Android are way too perfectly executed in iPhone and it DOES make a big difference in user experience!!2. The screen is AMAZING.. it doesn\\'t have 90 or 120 hz refresh rate but it FEELS better than my Nord with 90 hz!!3. A14 bionic promises years of performance ahead.4. Battery life is somewhere in between good & great.5.6. The dimensions of phone, it\\'s smaller than usual android phones and feels much more handy along with less weight.7. Overall build quality is great.8. Best audio output out there.9. Long term support.CONS:1